In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy import integrate
from scipy import special
import time
from tqdm import tqdm
import os
import sys

# Elastohydrodynamic Approach of Axisymmetric Indenter

This notebook implements a full numerical solver for the **axisymmetric elastohydrodynamic approach** of a deformable indenter toward a rigid surface, separated by a viscous fluid film.  
It generalizes the elastohydrodynamic bouncing model from [V. Bertin (2024)](https://github.com/vincent-bertin/elastohydrodynamic-bouncing) by allowing an indenter profile:

$$
h(r,t) = D(t) + \frac{r^{\text{exponent}}}{2} - w(r,t)
$$

where:
- $D(t)$ is the indenter position,
- $w(r,t)$ is its elastic deformation,
- $r^{\text{exponent}}/2$ defines the indenter shape.

The solver computes the **coupled evolution** of:
- fluid pressure $p(r,t)$,
- elastic deformation $w(r,t)$,
- indenter motion $D(t), V(t)$,
while enforcing lubrication flow and linear elasticity.



### Numerical and Simulation Parameters

This cell defines the core numerical settings used throughout the simulation:

- **Saving frequency**: every `N_save` timesteps the full radial fields are saved.
- **Parameter sweeps**: `Stokes_array` and `NR_array` list which Stokes numbers and radial resolutions can be run.
- **Active simulation parameters**: the Stokes number $St$, number of radial grid points `NR`, and the indenter shape exponent $r^{\text{exponent}}$ are selected.
- **Output directories**: folders are created to store global time series and radial field snapshots for the current simulation.

In [2]:
##########  NUMERICAL PARAMETERS ##########
# Saving Parameters
N_save = 100 # save full profile every N_save time step
Stokes_array = np.array([1000]) # Stokes numbers investigated
NR_array = np.array([2600]) # Number of radial grid points in each simulation

# External Parameters 
Stokes = 1000
NR = 2000

# Indenter shape
exponent = 1 


# Saving
os.mkdir('Stokes'+'%.0f' % Stokes +' exponent'+'%i' %exponent)
os.mkdir('Stokes'+'%.0f' % Stokes +' exponent'+'%i' %exponent +'/Fields')

### Spatial and Temporal Discretization

This cell defines the numerical grids in space and time:

- The radial domain spans $r \in [0, L_R]$ with a uniform grid spacing $\Delta r = L_R / NR$.
- Time is discretized with a constant timestep $\Delta t$, over a total of $N_T$ steps.
- An additional *initialization phase* of $N_{T,\mathrm{ini}}$ steps is introduced to gradually ramp the indenter velocity, avoiding a discontinuous pressure jump at $t=0$.

The arrays created here are:
- `$r$`: uniform radial coordinates,
- `Time_ini`: time axis for the initialization ramp,
- `Time`: time axis for the main simulation.

### Initialization Kinematics

The indenter begins at height $D_0 = 1$ and its position and velocity are prescribed during the initialization phase:

- $D_{\mathrm{ini}}(t) = D_0 - \left(t - \dfrac{t^{\text{exponent}}}{2 t_1}\right)$  
- $V_{\mathrm{ini}}(t) = -1 + \dfrac{t}{t_1}$

In [3]:
##########  AXIS DEFINITION ##########
# Space
LR = 6.0 # Domain size in r  [0, L]
dr = LR/NR # grid size

# Time axis.
dt = 0.001 # time increment - In the data use in the paper, we took dt = 1e-4 instead. 
NT = 10000 # number of time steps.

# Initialization
NT_ini = 100 # number of time steps in the initialization phase
t1 = -NT_ini*dt # t_ini in the Appendix.

# Axis
r = np.linspace(0, LR , NR) # We take a uniform distribution of point in r
Time_ini = np.linspace(t1, 0, NT_ini+1) # Initialization time axis.
Time = np.linspace(0, dt*NT, NT+1) # Time axis


########## Initial position ###########
D0 = 1
D_ini = D0 - (Time_ini-Time_ini**exponent/(2*t1)) # sphere position during initialization
V_ini = -1+Time_ini/t1 # sphere velocity during initialization

### Linear System Assembly and Time-Update Functions

This cell defines the core numerical operators used to advance the coupled elastohydrodynamic system in time.  
At each timestep, the unknown vector is

$$
y = \begin{pmatrix} w \\ p \end{pmatrix},
$$

of size $2\,NR$, containing both deformation $w(r)$ and pressure $p(r)$.

---

## 1. `LinearOperator()`: Builds the matrix $L$ in the linear system

$$
L(y^n)\, y^{n+1} = \mathrm{rhs}(y^n).
$$

This matrix couples:

### **Elasticity**
The first block enforces

$$
w_i = \sum_j K_{ij}\, p_j,
$$

where `KernelMatrix` contains the precomputed axisymmetric elastic Green’s function integrals.

---

### **Boundary conditions**

Enforces

$$
p_r(0)=0, \qquad p(R)=0,
$$

by modifying specific rows of $L$.

---

### **Lubrication thin-film equation**

The pressure block comes from the implicit discretization of

$$
-\dot w + V = \frac{1}{r}\,\partial_r \left( r\, h^3\, p_r \right),
$$

with film thickness

$$
h(r) = D + \frac{r^{\text{exponent}}}{2} - w(r).
$$

This produces a tridiagonal structure in the pressure rows:
- diagonal terms $\partial L_i / \partial p_i$,
- over- and under-diagonals $\partial L_i / \partial p_{i\pm1}$.

---

### **Coupling to deformation**

The terms $\partial L_i / \partial w_i$ bring in the implicit contribution from $-\dot{w}$.

---

## 2. `rhs()`: Builds the right-hand side vector

The explicit part of the thin-film equation gives

$$
\mathrm{rhs}_i = -\frac{(w_i + V\,\Delta t)\,\Delta r^2}{St\,\Delta t},
$$

applied to the pressure rows only.

---

## 3. `UpdateDistance_Velocity()`: Rigid-body motion update

Newton’s law for the indenter provides

$$
V^{n+1} = V^n + 4\,\Delta t \int_0^R p(r)\,r\,dr,
$$

and the new position is updated via

$$
D^{n+1} = D^n + V^{n+1}\,\Delta t.
$$

This step couples the hydrodynamic pressure back into the macroscopic motion of the indenter.

In [4]:
######### Main Function ###########
def LinearOperator(y_old, D_old, V_old, KernelMatrix):
    ''' Linear Matrix L of the linear system L * y = rhs.  '''
    L = np.zeros((2*NR,2*NR))
    w_old, p_old = y_old[:NR], y_old[NR:2*NR]



    ############## Elasticity  ##############
    np.fill_diagonal(L[:NR, :NR], np.ones(NR)) # L_{i,i}
    L[:NR, NR:2*NR] = KernelMatrix



    ############## Boundary conditions ##############
    L[NR,NR], L[NR,NR+1] = 1, -1
    L[2*NR-1,2*NR-1] = 1

    ############## Thin-film equation ##############
    h = D_old + r**exponent/2 - w_old
    alpha = 3 * (exponent/2*r[1:-1]**(exponent-1) - (w_old[2:]-w_old[1:-1])/dr ) + h[1:-1]/r[1:-1]
    diagonal_pressure, overdiagonal_pressure, underdiagonal_pressure = np.ones(NR-2), np.ones(NR-2), np.ones(NR-2)
    diagonal_pressure[:] = (2*h[1:-1]**3 + h[1:-1]**2 * alpha[:]*dr)
    overdiagonal_pressure[:] = -(h[1:-1]**3 + h[1:-1]**2 * alpha[:]*dr)
    underdiagonal_pressure[:] = -h[1:-1]**3

    np.fill_diagonal(L[NR+1:2*NR-1, NR+1:2*NR-1], diagonal_pressure) # L_{i,i} = partial L_i / partial p_i
    np.fill_diagonal(L[NR+1:2*NR-1, NR+2:2*NR-0], overdiagonal_pressure) # L_{i,i+1} = partial L_i / partial p_{i+1}
    np.fill_diagonal(L[NR+1:2*NR-1, NR+0:2*NR-2], underdiagonal_pressure) # L_{i,i-1} = partial L_i / partial p_{i-1}

    # Deformation terms
    diagonal_deformation = -dr**2 / (Stokes*dt)
    np.fill_diagonal(L[NR+1:2*NR-1, 1:NR-1], diagonal_deformation) # partial L_i / partial w_i

    return L





def rhs(y_old, V_old):
    ''' right hand side of the linear system L * y = rhs.  '''
    RHS = np.zeros(2*NR)
    w_old, p_old = y_old[:NR], y_old[NR:2*NR]
    RHS[NR+1:2*NR-2] = -(w_old[1:NR-2] + V_old*dt) * dr**2 / (Stokes*dt)
    return RHS



def UpdateDistance_Velocity(y_new, D_old, V_old):
    ''' Return the new velocity and distance '''
    w_new, p_new = y_new[:NR], y_new[NR:2*NR]
    V_new = V_old + 4 * dt * sum(p_new[:]*r[:]*dr)
    D_new = D_old + V_new*dt
    return D_new, V_new

<>:3: SyntaxWarning: invalid escape sequence '\c'
<>:3: SyntaxWarning: invalid escape sequence '\c'
/tmp/ipykernel_38612/2962282997.py:3: SyntaxWarning: invalid escape sequence '\c'
  ''' Linear Matrix L of the linear system L \cdot y = rhs.  '''


### Kernel Computation and Initialization Phase

The following cell performs two tasks:

---

## 1. Precompute the Elastic Kernel Matrix

The axisymmetric elastic deformation is given by the integral

$$
w(r_i) = -\frac{8}{\pi^2} \int_0^\infty 
\frac{u}{r_i + u}\,
K\!\left( \frac{4 r_i u}{(r_i + u)^2} \right)\,
p(u)\, du,
$$

where $K(\cdot)$ is the complete elliptic integral of the first kind.

Because evaluating this integral at every timestep would be extremely expensive, the notebook **precomputes** all integrals:

- Each entry `KernelMatrix[i, j]` stores the integral over one radial cell:
  - for $j=0$: integration over $u \in [0, \Delta r/2]$,
  - for $j>0$: integration over $[r_j - \Delta r/2,\, r_j + \Delta r/2]$.

This produces a dense matrix of size $NR \times NR$, allowing the deformation to be computed later as a simple matrix–vector product.

---

## 2. Initialization Sweep (Ramp-In Phase)

To avoid a discontinuity in the pressure at $t=0$, the simulation begins with an **initialization phase** of $NT_{\mathrm{ini}}$ timesteps.

During this phase, the solver advances

$$
y = \begin{pmatrix} w \\ p \end{pmatrix}
$$

using prescribed kinematics:
- The indenter position $D_{\mathrm{ini}}(t)$ moves smoothly from rest.
- The velocity $V_{\mathrm{ini}}(t)$ ramps from $0$ to $-1$.

At each initialization timestep:

1. The linear system

   $$
   L(y^n)\, y^{n+1} = \mathrm{rhs}(y^n)
   $$

   is solved for $y^{n+1}$ using `np.linalg.solve`.

2. The deformation and pressure fields are stored in  
   `Deformation_ini[:, i+1]` and `Pressure_ini[:, i+1]`.

The last initialization state then becomes the starting point for the main simulation.

In [5]:
print('Initialization : ')
KernelMatrix = np.zeros((NR, NR))
print('Kernel Computation')
for i in tqdm(range(0, NR)):
    KernelMatrix[i, 0] = scipy.integrate.quad(lambda u: u/(r[i]+u)*scipy.special.ellipk(4*r[i]*u/(r[i]+u)**2)/(np.pi**2/8) , 0, dr/2)[0]
    for j in range(1,NR):
        KernelMatrix[i, j] = scipy.integrate.quad(lambda u: u/(r[i]+u)*scipy.special.ellipk(4*r[i]*u/(r[i]+u)**2)/(np.pi**2/8) , r[j]-dr/2, r[j]+dr/2)[0]

# Initialization
y_ini, Deformation_ini, Pressure_ini = np.zeros((2*NR, NT_ini)), np.zeros((NR, NT_ini)), np.zeros((NR, NT_ini))
for i in tqdm(range(0, NT_ini-1)):
    y_ini[:,i+1] = np.linalg.solve(LinearOperator(y_ini[:,i], D_ini[i], V_ini[i], KernelMatrix), rhs(y_ini[:,i], V_ini[i]))
    Deformation_ini[:,i+1], Pressure_ini[:,i+1] = y_ini[:NR,i+1], y_ini[NR:2*NR,i+1]
print('Initialization == DONE')

Initialization : 
Kernel Computation


  4%|█▌                                       | 78/2000 [00:04<01:54, 16.76it/s]/tmp/ipykernel_38612/3259590497.py:7: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  KernelMatrix[i, j] = scipy.integrate.quad(lambda u: u/(r[i]+u)*scipy.special.ellipk(4*r[i]*u/(r[i]+u)**2)/(np.pi**2/8) , r[j]-dr/2, r[j]+dr/2)[0]
100%|███████████████████████████████████████████| 99/99 [01:35<00:00,  1.03it/s]

Initialization == DONE


### Main Time-Integration Loop

This cell advances the coupled elastohydrodynamic system in time, updating deformation $w(r,t)$, pressure $p(r,t)$, and the indenter motion $D(t), V(t)$.

---

## 1. Allocate State and Output Arrays

The unknown vector is

$$
y = \begin{pmatrix} w \\ p \end{pmatrix},
$$

stored for all timesteps in the array `y`.  
The arrays `Deformation` and `Pressure` store $w(r,t)$ and $p(r,t)$ separately.

Initial values are copied from the final state of the initialization phase.

Global quantities such as force, elastic energy, and film thickness are also allocated.

---

## 2. Time-Stepping Loop

For each timestep $i$:

### (a) Solve the implicit system

$$
L(y^n)\,y^{n+1} = \mathrm{rhs}(y^n),
$$

computed with:

y[:, i+1] = np.linalg.solve( LinearOperator(...), rhs(...) )

### (b) Extract the updated fields

Deformation[:, i+1] = y[:NR, i+1]
Pressure[:, i+1] = y[NR:, i+1]


---

### (c) Update indenter motion

Rigid-body dynamics give:

$$
V^{n+1} = V^n + 4\,\Delta t \int_0^R p(r)\,r\,dr,
$$

$$
D^{n+1} = D^n + V^{n+1}\Delta t.
$$

---

### (d) Compute diagnostics

- Elastic energy  
  $$
  E_{\mathrm{el}} = -2\int_0^R p(r)w(r)\,r\,dr.
  $$

- Hydrodynamic force  
  $$
  F = 4\int_0^R p(r)\,r\,dr.
  $$

- Central film thickness  
  $$
  h(0,t) = D(t) - w(0,t).
  $$

- Minimum film thickness  
  $$
  h_{\min}(t) = \min_r\left[D(t) + \frac{r^{\text{exponent}}}{2} - w(r,t)\right].
  $$

---

## 3. Saving Results

At every timestep, global quantities $(t, D, V, F, E_{\mathrm{el}}, h(0), h_{\min})$ are saved.  
Every `N_save` timesteps, full radial fields $(r, w, p)$ are written to disk.

---

## 4. Stopping Condition

The loop terminates when the indenter begins to rebound:

$$
V(t) > 0.
$$

This marks the end of the approach phase.


In [6]:
y, Deformation, Pressure = np.zeros((2*NR, NT+1)), np.zeros((NR, NT+1)), np.zeros((NR, NT+1))
Deformation[:,0], Pressure[:,0] = Deformation_ini[:,-1], Pressure_ini[:,-1]
y[:NR, 0], y[NR:, 0] = Deformation[:,0], Pressure[:,0]
D, V, Force, EnergyElastic, Central_Film_thikness, MinFilmThickness = np.zeros(NT+1), np.zeros(NT+1), np.zeros(NT+1), np.zeros(NT+1), np.zeros(NT+1), np.zeros(NT+1)
D[0], V[0] = D_ini[-1], V_ini[-1]
print('Main loop : ')
for i in tqdm(range(0, NT)):
    y[:,i+1] = np.linalg.solve(LinearOperator(y[:,i], D[i], V[i], KernelMatrix), rhs(y[:,i], V[i]))
    Deformation[:,i+1], Pressure[:,i+1] = y[:NR,i+1], y[NR:2*NR,i+1]
    D[i+1], V[i+1] = UpdateDistance_Velocity(y[:,i+1], D[i], V[i])
    EnergyElastic[i+1] = -2*np.sum(Pressure[:,i+1]*Deformation[:,i+1]*r[:]*dr)
    Force[i+1] = 4*np.sum(Pressure[:,i+1]*r[:]*dr)
    Central_Film_thikness[i+1] = D[i+1] - Deformation[0,i+1]
    MinFilmThickness[i+1] = min(D[i+1] + r[:]**exponent/2 - Deformation[:,i+1])

    #Saving
    np.savetxt('Stokes'+'%.0f' % Stokes +' exponent'+'%i' %exponent +'/Global_St'+'%.0f' % Stokes+'_coarse_timestep.txt', np.c_[Time[:i+1], D[:i+1], V[:i+1], Force[:i+1], EnergyElastic[:i+1], Central_Film_thikness[:i+1], MinFilmThickness[:i+1]], delimiter=' ',fmt=['%-.9g', '%-.9g', '%-.9g', '%-.9g', '%-.9g', '%-.9g', '%-.9g'])
    if i%N_save == 0:
#         print(np.mean(Pressure[:,i]))
        np.savetxt('Stokes'+'%.0f' % Stokes +' exponent'+'%i' %exponent+'/Fields/t'+'%.3f' % Time[i]+'_coarse_timestep.txt', np.c_[r, Deformation[:,i], Pressure[:,i]], delimiter=' ',fmt=['%-.9g', '%-.9g', '%-.9g'])
    if V[i+1] > 0:
        break

print('Contact == DONE')

Main loop : 


  0%|                                      | 20/10000 [00:20<2:50:55,  1.03s/it]


KeyboardInterrupt: 